In [1]:
import imp
import numpy as np
import pandas as pd
import openpyxl
import os 
import xlrd
# Setting Directory
os.chdir('C:/Users/TAM/Documents/Vu Quang Nguyen/VACP/')

C:\Users\TAM\AppData\Local\Temp\ipykernel_8248\2202898749.py:1: DeprecationWarning: the imp module is deprecated in favour of importlib and slated for removal in Python 3.12; see the module's documentation for alternative uses
  import imp


In [2]:
# INPUT 
data_input=pd.read_excel("../VACP/Python NCC Min/Analysing Qty 4 Months/Input.xlsx",sheet_name="Input").rename(columns={"FOB Price (USD/Pcs)":"Unit Price"})
# Exchange Rate
data_exchange_rate=pd.read_excel("../VACP/Python NCC Min/Vietcombank Exchange Currency.xlsx",usecols="B,E").rename(columns={"Giá bán VND":"Tỷ giá VND"})

# # Tinh Unit Price VND
data_input=data_input.merge(data_exchange_rate,on="Mã nguyên tệ")
Exchange_Cost=0.05

data_input["Unit Price VND"] = np.round((1+Exchange_Cost)*data_input["Unit Price"]*data_input["Tỷ giá VND"],0)
data_input=data_input.drop(columns=["No.","Tỷ giá VND"])
data_input.head()


,Brand,Order Conf. Date,Calculated Date,Order Ref.,PO Number,ID,Substitution,Description,Order Qty,Unit Price,Mã nguyên tệ,Total Order Amount,Agt 22 Ready Stock,Số lượng cần nhập,Số lượng chênh lệch,Note,Unit Price VND
0,Denso,2022-01-01,2022-08-22,NaN,NaN,065000-2940ND AC,NaN,Motor Assy,20,31.85,USD,637.00,0.0,20.0,0.0,Order,803623.0
1,Denso,2022-01-01,2022-08-22,NaN,NaN,168000-1070ND AC,NaN,Motor Fan,20,18.35,USD,367.00,0.0,20.0,0.0,Order,462998.0
2,Denso,2022-01-01,2022-08-22,NaN,NaN,291000-9011 ND,NaN,Fuel Pump,15,24.99,USD,374.85,0.0,20.0,-5.0,Order,630535.0
3,Denso,2022-01-01,2022-08-22,NaN,NaN,291000-9011 ND,NaN,Fuel Pump,20,24.99,USD,499.80,0.0,20.0,0.0,Order,630535.0
4,Toyota,2022-04-26,2022-08-22,NaN,NaN,13011-0L030,NaN,"RING SET, PISTON",4,45.83,USD,183.32,0.0,4.0,0.0,Order,1156360.0


In [3]:
# NHAP - XUAT - TON  * GIA BAN * DANH MUC HANG HOA

data_nhapxuatton=pd.read_excel("../VACP/Python NCC Min/Tổng hợp Nhập - Xuất - Tồn YTD.xls",usecols="A:B,G:O",skipfooter=1,header=5).rename(columns={
                                "Unnamed: 0":"ID",
                                "Unnamed: 1":"Tên vật tư",
                                "Số lượng":"Tổng đầu kỳ", 
                                "Giá trị":"Giá trị đầu kỳ",
                                "Số lượng.1":"Tổng nhập", 
                                "Giá trị.1":"Giá trị nhập",
                                "Số lượng.2":"Tổng xuất", 
                                "Giá trị.2":"Giá trị xuất",
                                "Số lượng.3":"Tổng tồn", 
                                "Giá trị.3":"Giá trị tồn" }).dropna(subset=['ID']).reset_index(drop=True)


data_banhang=pd.read_excel("../VACP/Bao cao Ban hang Power BI/Sổ chi tiết bán hàng.xls",usecols="A,C:D,F,H,K:M",skipfooter=1,header=6).rename(columns={
                                "Unnamed: 0":"Ngày bán",
                                "Unnamed: 2":"ID Khách hàng",
                                "Unnamed: 3":"Hãng",
                                "Unnamed: 5":"ID",
                                "Unnamed: 7":"Tên hàng",
                                "Số lượng":"Tổng bán", 
                                "Đơn giá gốc":"Giá bán",
                                "Giá bán":"Giá bán giảm"
                                }).dropna(subset=['ID']).reset_index(drop=True)
                                
data_giaban=pd.read_excel("../VACP/Python NCC Min\Báo cáo giá bán, tồn kho.xls",usecols="C,F",skipfooter=1,header=4).rename(columns={"Mã số":"ID"}).dropna(
                subset=['ID']).reset_index(drop=True)

data_danhmuchanghoa=pd.read_excel("../VACP/Python NCC Min/Danh mục vật tư hàng hóa .xls",usecols="A,C:D,M").rename(columns={
                                "Unnamed: 0":"ID Nhóm hàng",
                                "Unnamed: 2":"ID",
                                "Unnamed: 3":"Tên vật tư",
                                "Unnamed: 12":"Hãng"
                                }).dropna(subset=['ID','ID Nhóm hàng']).reset_index(drop=True)



C:\Users\TAM\AppData\Local\Temp\ipykernel_8248\1085106072.py:3: FutureWarning: Defining usecols with out of bounds indices is deprecated and will raise a ParserError in a future version.
  data_nhapxuatton=pd.read_excel("../VACP/Python NCC Min/Tổng hợp Nhập - Xuất - Tồn YTD.xls",usecols="A:B,G:O",skipfooter=1,header=5).rename(columns={


In [ ]:
# Giá vốn
TransCost=0.05
data_nhapxuatton['Giá vốn'] =np.round(
                                         (1+TransCost)*np.where(data_nhapxuatton["Giá trị xuất"]>0,
                                        data_nhapxuatton["Giá trị xuất"]/data_nhapxuatton["Tổng xuất"],
                                            np.where(data_nhapxuatton["Giá trị tồn"]>0,
                                                data_nhapxuatton["Giá trị tồn"]/data_nhapxuatton["Tổng tồn"],0

                                            ))
                                    ,0)

In [ ]:
#Bán hàng gần đây
max_date_banhang=data_banhang.groupby(["ID"])["Ngày bán"].max()
data_banhang_latest=data_banhang.merge(max_date_banhang,on="ID",how="left",suffixes=('', '_max'))
data_banhang_latest=data_banhang_latest[data_banhang_latest["Ngày bán"]==data_banhang_latest["Ngày bán_max"]].drop('Ngày bán_max',axis=1).drop_duplicates(subset="ID",keep='first').reset_index(drop=True)
 

In [ ]:
# Gộp dữ liệu
data_nhap_ban=data_nhapxuatton.merge(data_danhmuchanghoa[["ID","Hãng"]],on="ID").merge(data_banhang_latest[["ID","Ngày bán","Giá bán giảm"]],on='ID').merge(data_giaban,on="ID")
# TÍnh số tháng tồn trong kho


data_nhap_ban["Số tháng tồn trong kho"]=np.round(
                                        (363/30)/data_nhap_ban["Giá vốn"]/(np.where(
                                        # Nếu Tồn dương và đầu kỳ dương thì lấy Giá vốn / (TB Giá trị đầu kỳ + nhập + Cuối kỳ)
                                                        (data_nhap_ban["Giá trị đầu kỳ"]>0) & (data_nhap_ban["Giá trị tồn"]>0),
                                                                np.mean([data_nhap_ban["Tổng đầu kỳ"],data_nhap_ban["Tổng tồn"]])/
                                                                    np.mean([data_nhap_ban["Giá trị đầu kỳ"],data_nhap_ban["Giá trị tồn"]]),-999)),1)
                                
data_report=data_input.merge(data_nhap_ban,on="ID",how="left")
# data_report=data_input.merge(data_nhap_ban[["ID","Giá vốn","Giá bán","Giá bán giảm","Tổng đầu kỳ","Tổng nhập","Tổng xuất","Tổng tồn","Số tháng tồn trong kho"]], on="ID", how="left")



In [ ]:
data_report=data_report[["ID",
    "Order Conf. Date",
    "Calculated Date",
    "Unit Price VND",
    "Giá vốn",
    "Giá bán giảm", 
    "Số lượng cần nhập", 
    "Agt 22 Ready Stock",
    "Tổng đầu kỳ",
    "Tổng nhập",
    "Tổng xuất",
    "Tổng tồn",
    "Giá trị đầu kỳ", 
    "Giá trị nhập",
    "Giá trị xuất", 
    "Giá trị tồn",
    "Số tháng tồn trong kho"]].sort_values(by=["Số tháng tồn trong kho","Tổng tồn","ID"],ascending=False).reset_index(drop=True)

In [ ]:
# EXPORT FILE DATA
with pd.ExcelWriter('../VACP/Python NCC Min/Analysing Qty 4 Months/Report.xlsx') as writer:
    data_report.to_excel(writer,sheet_name="Tổng hợp")
